In [1]:
import os

import getpass

from dotenv import load_dotenv


load_dotenv()

True

In [ ]:
import os

import getpass

load_dotenv()


if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [3]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# all_splits = text_splitter.split_documents(docs)

# # Index chunks
# _ = vector_store.add_documents(documents=all_splits)


from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


load_dotenv()


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


def extract_data():
    text_chunks = []
    files = filter(lambda f: f.lower().endswith(".pdf"), os.listdir("./content_source/pdf"))
    file_list = list(files)
    for file in file_list:
        loader = PyPDFLoader(os.path.join('content_source', 'pdf', file))
        text_chunks += loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter(
            chunk_size = 512,
            chunk_overlap = 30,
            length_function = len,
            separators= ["\n\n", "\n", ".", " "]
        ))
    vectorstore = FAISS.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY))
    return vectorstore


vector_store = extract_data()




# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# response = graph.invoke({"question": "What is Task Decomposition?"})
response = graph.invoke({"question": "What is this content all about?"})
print(response["answer"])

The content discusses advancements in machine learning and deep learning applications within the context of Directed Energy Deposition (DED) additive manufacturing. It focuses on using image analysis of melt pools to predict the quality of printed components, specifically by estimating deviations in printing focus. The work aims to enhance prediction accuracy by incorporating temporal information from preceding images.


In [6]:
response

{'question': 'What is this content all about?',
 'context': [Document(id='d92c7eed-4701-44f8-b069-301d436a9359', metadata={'producer': 'Adobe PSL 1.2e for Canon\x00', 'creator': 'Canon iR-ADV C5250  PDF', 'creationdate': '2025-08-04T12:26:55+03:00', 'source': 'content_source/pdf/projeto_leonel-rev_kan1.pdf', 'total_pages': 39, 'page': 29, 'page_label': '30'}, page_content='58'),
  Document(id='f17b0ab1-2940-4380-94b7-31273af7e9ba', metadata={'producer': 'Adobe PSL 1.2e for Canon\x00', 'creator': 'Canon iR-ADV C5250  PDF', 'creationdate': '2025-08-04T12:26:55+03:00', 'source': 'content_source/pdf/projeto_leonel-rev_kan1.pdf', 'total_pages': 39, 'page': 38, 'page_label': '39'}, page_content='Elgendy, Mohamed. Deep learningfor vision systems. Simon and Schuster, 2020.\nEra, Israt Zarin, et a1.,2023. "Machine learning in Directed Energy Deposition (DED) additive manufacturing: A state-\noÊthe-art review." Manufacturing Letters 35, pp. 689-700.\nJogin, Manjunath,etal.,20l8. "Feature extract